<table>
  <tr>
    <td><img src="https://github.com/rvss-australia/RVSS/blob/main/Pics/RVSS-logo-col.med.jpg?raw=1" width="400"></td>
    <td><div align="left"><font size="30">Spatial Operators</font></div></td>
  </tr>
</table>

(c) Peter Corke 2024

Robotics, Vision & Control: Python, see Chapter 11

## Configuring the Jupyter environment
We need to import some packages to help us with linear algebra (`numpy`), graphics (`matplotlib`), and machine vision (`machinevisiontoolbox`).
If you're running locally you need to have these packages installed.  If you're running on CoLab we have to first install machinevisiontoolbox which is not preinstalled, this will be a bit slow.

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False

%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np
from machinevisiontoolbox import *

# display result of assignments
if COLAB:
    %config ZMQInteractiveShell.ast_node_interactivity = 'last_expr_or_assign'
# make NumPy display a bit nicer
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:10.4g}" if abs(x) > 1e-10 else f"{0:10.4g}"})



Now let's load a real greyscale image from a PNG file.  This particular image file is distributed with the Toolbox, but you can pass in the path to any image file you might have.  _If the Toolbox can't find the specified image it defaults to looking in the folder of images distributed with the Toolbox._

In [ ]:
mona = Image.Read("monalisa.png", mono=True)  # convert to monochrome
mona.disp()

The image of the Mona Lisa looks rather grainy, the paint is cracked, but then she is over 500 years old...

# Image smoothing by averaging

We could smooth it out by local averaging, where every pixel in the output image is the average of all pixels in a $5 \times 5$ window about the corresponding input pixel.  We first create a kernel

In [ ]:
kernel = np.ones((5,5),np.float32) / 25
kernel

where the sum of all values is equal to one.  For a given input pixel, say (20,30), the $5 \times 5$ window is

In [ ]:
window = mona[20-2:20+3,30-2:30+3].image
window

<p style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">Note that the range 20-2:20+3 includes 20-2, 20-1, 20-0, 20+1, 20+2.  In Python the end value of  a range is not included in the set.</p>

In [ ]:
z = window * kernel  # using element-wise multiplication
z

In [ ]:
np.sum(z)

and this is the value of pixel (20,30) in the output image.  

We need do this for every pixel in the image, and we could use a pair of nested `for` loops but that's not very efficient.  We can use optimised code in OpenCV

In [ ]:
smoothed = mona.convolve(kernel)
smoothed.disp();

**Q: Vary the dimensions of the kernel to see what effect it has?**

The square window can introduce artifact in the image, horizontal lines induced by contrast steps.  When `n` is large the monalisa develops a moustache!

The problem is because the output pixel is a weighted sum of input pixels in the window, and all pixels are weighted equally irrespective of their distance from the central pixel.  We can solve this by using a symmetric kernel.

## Gaussian blur

For image smoothing it is preferable to use a kernel that is isotropic and symmetric such as a 2D Gaussian

$G(u,v) = \frac{1}{2\pi\sigma^2}e^{-\frac{u^2+v^2}{2\sigma^2}}$

  We will create a 2D Gaussian with $\sigma=10$

In [ ]:
K = Kernel.Gauss(10)
print(K)

We can display the kernel as a 3-dimensional mesh.

In [ ]:
K.disp3d()

Now we can convolve the image with this kernel

In [ ]:
mona.convolve(K).disp();

We can do this more simply using the `smooth` method which accepts $\sigma$ and the half-kernel width.

In [ ]:
mona.smooth(2, 5).disp();

**Q: Modify the code block above and look at the effect of changing $\sigma$**

## Finding edges
We can use 2D filtering to find edges as well.  This convolution kernel will find vertical edges.  The intuition is that each row of this kernel subtracts the pixel to the left from the pixel to the right, which will give a positive value if the intensity is increasing left to right.

In [ ]:
kernel = np.array( [ [1, 0, -1],
                     [2, 0, -2],
                     [1, 0, -1] ]) / 8

<p style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">You may often see this filter kernel written with the first and third columns swapped.  This is appropriate if you perform correlation, not convolution. These are two similar operations but differ in the kernel being reflected about its centre.  Many kernels are symmetric which means that convolution and correlation are the same.</p>

In [ ]:
penguins = Image.Read('penguins.png', grey=True, dtype='float')
penguins.disp();

In [ ]:
gradient_horizontal = penguins.convolve(kernel)
gradient_horizontal.disp(colormap='signed');                

The image is displayed with a color map that shows negative numbers as red and positive numbers as blue.  Zoom in on the outline of the "P" (use the second button from the right in the bottom toolbar) and you can see that the intensity goes up (blue) on the left side of the "P", from the grey background to the white paint. It goes down (red) on right of the stem, from the white paint to the gray background.

<p style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">Our original image comprised unsigned integers (uint8) which are unable to express negative values.  The output of convolve() is a signed floating point image meaning the result at each pixel, can be positive or negative.</p>

We can find the horizontal edges by finding vertical gradient, using the transpose of the kernel

In [ ]:
gradient_vertical = penguins.convolve(kernel.T)
gradient_vertical.disp(colormap='signed');    